# RO47019: Intelligent Control Systems Practical Assignment
* Period: 2022-2023, Q3
* Course homepage: https://brightspace.tudelft.nl/d2l/home/500969
* Instructor: Cosimo Della Santina (C.DellaSantina@tudelft.nl)
* Teaching assistant: Ruben Martin Rodriguez (R.MartinRodriguez@student.tudelft.nl)
* (c) TU Delft, 2023

Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. Remove `raise NotImplementedError()` afterwards. Moreover, if you see an empty cell, please DO NOT delete it, instead run that cell as you would run all other cells. Please fill in your name(s) and other required details below:

In [ ]:
# Please fill in your names, student numbers, netID, and emails below.
STUDENT_1_NAME = ""
STUDENT_1_STUDENT_NUMBER = ""
STUDENT_1_NETID = ""
STUDENT_1_EMAIL = ""

In [ ]:
# Note: this block is a check that you have filled in the above information.
# It will throw an AssertionError until all fields are filled
assert STUDENT_1_NAME != ""
assert STUDENT_1_STUDENT_NUMBER != ""
assert STUDENT_1_NETID != ""
assert STUDENT_1_EMAIL != ""

### General announcements

* Do *not* share your solutions, and do *not* copy solutions from others. By submitting your solutions, you claim that you alone are responsible for this code.

* Do *not* email questions directly, since we want to provide everybody with the same information and avoid repeating the same answers. Instead, please post your questions regarding this assignment in the correct support forum on Brightspace, this way everybody can benefit from the response. If you do have a particular question that you want to ask directly, please use the scheduled Q&A hours to ask the TA.

* There is a strict deadline for each assignment. Students are responsible to ensure that they have uploaded their work in time. So, please double check that your upload succeeded to the Brightspace and avoid any late penalties.

* This [Jupyter notebook](https://jupyter.org/) uses `nbgrader` to help us with automated tests. `nbgrader` will make various cells in this notebook "uneditable" or "unremovable" and gives them a special id in the cell metadata. This way, when we run our checks, the system will check the existence of the cell ids and verify the number of points and which checks must be run. While there are ways that you can edit the metadata and work around the restrictions to delete or modify these special cells, you should not do that since then our nbgrader backend will not be able to parse your notebook and give you points for the assignment. You are free to add additional cells, but if you find a cell that you cannot modify or remove, please know that this is on purpose.

* This notebook will have in various places a line that throws a `NotImplementedError` exception. These are locations where the assignment requires you to adapt the code! These lines are just there as a reminder for youthat you have not yet adapted that particular piece of code, especially when you execute all the cells. Once your solution code replaced these lines, it should accordingly *not* throw any exceptions anymore.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

# Problem 1 - Vision-based angle prediction (42.5p)

**Authors:** Tomás Coleman (T.Coleman@tudelft.nl), Chuhan Zhang (C.Zhang-8@tudelft.nl)

Run the following cells before starting the tasks

In [ ]:
# Reloads the python files outside of this notebook automatically
%load_ext autoreload
%autoreload 2

# import all Python modules
from distutils.util import strtobool
from functools import partial
from IPython.display import display, HTML  # For animations in the notebook
from jax.config import config as jax_config

jax_config.update("jax_platform_name", "cpu")  # set default device to 'cpu'
jax_config.update("jax_enable_x64", True)  # double precision
import jax
from jax import jit, lax, random
from jax import numpy as jnp
import numpy as onp
import cv2
from matplotlib import rcParams
import matplotlib.pyplot as plt
import os
from pathlib import Path
from progressbar import progressbar
from typing import Dict, Tuple
from scipy import stats

from jax_double_pendulum.robot_parameters import ROBOT_PARAMS
from jax_double_pendulum.robot_simulation import simulate_robot
from jax_double_pendulum.utils import normalize_link_angles

import torch
from torch.utils.data import Dataset, DataLoader
import snntorch as snn
from snntorch import surrogate
import torch.nn as nn
import torch.nn.functional as F
import os
from utils import *


# define boolean to check if the notebook is run for the purposes of autograding
AUTOGRADING = strtobool(os.environ.get("AUTOGRADING", "false"))
""
# create directory for datasets
datasets_dir = Path("datasets")
datasets_dir.mkdir(parents=True, exist_ok=True)

# create directory for plots
outputs_dir = Path("outputs")
outputs_dir.mkdir(parents=True, exist_ok=True)

# create directory for state dictionaries of neural network
statedicts_dir = Path("statedicts")
statedicts_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
class ThetaDataset(Dataset):
    def __init__(self, dataset):
        self.x = torch.tensor(dataset["th_pix_curr"], dtype=torch.float32) / 255.0
        self.y = torch.tensor(dataset["th_curr_ss"], dtype=torch.float32)

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]

        return x, y

    def __len__(self):
        num_samples = self.x.shape[0]

        return num_samples


class TrigDataset(Dataset):
    def __init__(self, dataset):
        self.x = torch.tensor(dataset["th_pix_curr"], dtype=torch.float32) / 255.0
        y_cos = torch.cos(torch.tensor(dataset["th_curr_ss"], dtype=torch.float32))
        y_sin = torch.sin(torch.tensor(dataset["th_curr_ss"], dtype=torch.float32))
        self.y = torch.cat((y_sin, y_cos), dim=-1)

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]

        return x, y

    def __len__(self):
        num_samples = self.x.shape[0]

        return num_samples


class CNNDataset(Dataset):
    def __init__(self, dataset):
        self.x = (
            torch.tensor(
                onp.transpose(dataset["th_pix_curr"], (0, 3, 1, 2)), dtype=torch.float32
            )
            / 255.0
        )
        y_cos = torch.cos(torch.tensor(dataset["th_curr_ss"], dtype=torch.float32))
        y_sin = torch.sin(torch.tensor(dataset["th_curr_ss"], dtype=torch.float32))
        self.y = torch.cat((y_sin, y_cos), dim=-1)

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        return x, y

    def __len__(self):
        num_samples = self.x.shape[0]

        return num_samples


class SNNDataset(Dataset):
    def __init__(self, path, num_itr, num_data):
        self.path = path
        self.num_itr = num_itr
        self.num_data = num_data

    def __len__(self):
        return int(self.num_itr * self.num_data)

    def __getitem__(self, index):
        if index < int(self.num_data * self.num_itr):
            spike_path = "spike" + str(index) + ".pt"
            label_path = "target" + str(index) + ".pt"
            spike_out = torch.load(os.path.join(self.path, spike_path))
            label_out = torch.load(os.path.join(self.path, label_path))
            return spike_out, label_out
        else:
            raise IndexError

In [ ]:
def load_training_dataset_dataloader(
    _filepath, _rng: random.KeyArray, _val_ratio=0.3, batch_size=64, model_type="theta"
):
    assert 0.0 <= _val_ratio <= 1.0, "Validation ratio needs to be in interval [0, 1]."

    _dataset = jnp.load(_filepath)
    num_samples = _dataset["th_curr_ss"].shape[0]

    indices = jnp.arange(num_samples)
    shuffled_indices = random.permutation(_rng, indices)
    num_train_samples = int((1 - _val_ratio) * num_samples)
    split_config = jnp.array(
        [
            num_train_samples,
        ]
    )
    train_indices, val_indices = jnp.split(shuffled_indices, split_config)

    _train_ds, _val_ds = {}, {}
    for key, val in _dataset.items():
        _train_ds[key] = val[train_indices]
        _val_ds[key] = val[val_indices]

    if model_type == "theta":
        train_data = ThetaDataset(_train_ds)
        val_data = ThetaDataset(_val_ds)
    elif model_type == "trig":
        train_data = TrigDataset(_dataset)
        val_data = TrigDataset(_val_ds)
    elif model_type == "cnn":
        train_data = CNNDataset(_dataset)
        val_data = CNNDataset(_val_ds)

    _train_dataloader = torch.utils.data.DataLoader(
        train_data, batch_size=batch_size, shuffle=True
    )
    _val_dataloader = torch.utils.data.DataLoader(
        val_data, batch_size=batch_size, shuffle=True
    )

    return _train_dataloader, _val_dataloader


def load_test_dataset_dataloader(
    _filepath, _rng: random.KeyArray, batch_size=64, model_type="theta"
):
    _dataset = jnp.load(_filepath)
    num_samples = _dataset["th_curr_ss"].shape[0]

    if model_type == "theta":
        print("Theta Dataset")
        _data = ThetaDataset(_dataset)

    elif model_type == "trig":
        print("Trig Dataset")
        _data = TrigDataset(_dataset)
    elif model_type == "cnn":
        print("CNN dataset")
        _data = CNNDataset(_dataset)

    _dataloader = torch.utils.data.DataLoader(
        _data, batch_size=batch_size, shuffle=False
    )
    return _dataloader, _dataset

In [ ]:
def evaluate_model(_model, _test_dataset, model="theta", file="nothing.pdf"):
    filepath = str(outputs_dir / file)
    bin_no = 10
    _theta = _test_dataset["th_curr_ss"]
    heatmap, xedges, yedges = onp.histogram2d(_theta[:, 0], _theta[:, 1], bins=bin_no)
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

    if model == "cnn":
        x = onp.transpose(
            onp.array(_test_dataset["th_pix_curr"], dtype=onp.float32), (0, 3, 1, 2)
        )
        y_hat = _model(torch.tensor(x)).detach().numpy()
    else:
        y_hat = (
            _model(
                torch.tensor(onp.array(_test_dataset["th_pix_curr"], dtype=onp.float32))
            )
            .detach()
            .numpy()
        )  # need to conver back to np array

    if model == "theta":
        y = _test_dataset["th_curr_ss"]
    else:
        y_cos = onp.cos(_test_dataset["th_curr_ss"])
        y_sin = onp.sin(_test_dataset["th_curr_ss"])
        y = onp.concatenate((y_cos, y_sin), axis=-1)

    error = y_hat - y
    error = onp.absolute(error)
    ind_th1 = onp.digitize(_test_dataset["th_curr_ss"][:, 0], xedges, right=True)
    ind_th2 = onp.digitize(_test_dataset["th_curr_ss"][:, 1], yedges, right=True)

    ind_x = []
    ind_y = []
    for i in range(bin_no):
        ind_x.append(onp.where(ind_th1 == (i + 1))[0])
        ind_y.append(onp.where(ind_th2 == (i + 1))[0])

    avg_bins = onp.zeros((bin_no, bin_no))
    for i in range(bin_no):
        for j in range(bin_no):
            avg_bins[i, j] = (
                onp.mean(
                    onp.concatenate(
                        (error[onp.array(ind_x[i]), 0], error[onp.array(ind_y[j]), 1])
                    )
                )
                / heatmap[i, j]
            )

    plt.clf()

    plt.imshow(avg_bins, extent=extent)
    plt.title("Heatmap of prediction error")
    plt.colorbar()
    plt.savefig(filepath)
    plt.show()


def trig_to_theta(trig_data):
    trig_len = trig_data.shape[0]
    theta = torch.zeros(trig_len, 2)
    for i in range(trig_len):
        theta[i, 0] = torch.atan2(trig_data[i, 0], trig_data[i, 2])
        theta[i, 1] = torch.atan2(trig_data[i, 1], trig_data[i, 3])

    return theta

In [ ]:
# set seed for jax
rng = jax.random.PRNGKey(seed=42)
rng, init_rng = jax.random.split(rng)

## Task 1.1: Learn to predict angles (7.5p)

We are going to create models that try to predict the link angles $\hat{\theta}$ given an image of the robot, so $\theta \approx \hat{\theta} =M(x)$.

### 1.1 - Prepare data 
Run the functions below to load the data generated in `task_1a_generate_data.ipynb`, split the training into train/validation and put them in a dataloader. 

In [ ]:
if not AUTOGRADING:
    train_loader_theta, val_loader_theta = load_training_dataset_dataloader(
        str(datasets_dir / "dataset_double_pendulum_train.npz"),
        _rng=init_rng,
        model_type="theta",
    )
    test_loader_theta, test_dataset_theta = load_test_dataset_dataloader(
        str(datasets_dir / "dataset_double_pendulum_test.npz"),
        _rng=init_rng,
        model_type="theta",
    )

### 1.1 - Create the model (1.5p)
Here, create a sequaential PyTorch model class called NeuralNetworkTheta()
- Start by flattening the input with `torch.flatten()`.
- Then, add a fully connected hidden layer `torch.nn.Linear(...)` of 128 units and ReLU activation.
- Finally, add a final fully connected linear layer `torch.nn.Linear(...)`without activation. The number of units must match the dimension of our target data, which is 2, as we try to predict the angles $\theta_1$ and $\theta_2$.


In [ ]:
"""
NN Model with Theta as the output
"""

""" TASK 1.1: CREATE MODEL HERE """

# YOUR CODE HERE
raise NotImplementedError()
"""TASK1.1: END"""
model_theta = NeuralNetworkTheta()

total_params_theta = sum(p.numel() for p in model_theta.parameters())
print("total number of model parameters: ", total_params_theta)

### 1.1 - Train the model (2p) 

Now that the model is defined, we can train it using the training dataset. We have a train-validation split of 0.3 and a batch size of 64 that is shuffled every epoch. Train the model for 50 epochs with the training data `train_loader_theta`. Do this 10 times and record the prediction error of the final model of each of the 10 runs on the test dataset `test_loader_theta` using the function `evaluate_model_test_data_theta`. Don't forget to reinitialize the parameters on each run! **hint:** reduce the number of epochs and lower the number of runs to `1` while getting your model working. 

In [ ]:
# Training parameters
optimizer = torch.optim.SGD(model_theta.parameters(), lr=1e-3)
# We choose the mean square loss as it is a regression problem
loss_fn = torch.nn.MSELoss()
error_fn = torch.nn.L1Loss()
num_epochs = 100


def evaluate_model_test_data_theta(_model, _test_loader):
    running_loss_test = 0.0
    running_error_test = 0.0
    count = 0
    with torch.no_grad():
        for i, data_test in enumerate(_test_loader):
            count += 1
            inputs_test, labels_test = data_test
            batch = labels_test.shape[0]
            est_th = _model(inputs_test)
            loss_test = loss_fn(est_th, labels_test)
            error_test = error_fn(est_th, labels_test)
            running_loss_test += loss_test.item()
            running_error_test += error_test.item()
    print(
        f"Loss on test data: {running_loss_test / count:.3f}, Average prediction error of model on test data: {running_error_test / count:.3f}"
    )
    return running_error_test / count

In [ ]:
if not AUTOGRADING:
    """TASK 1.1: TRAIN MODEL HERE"""
    # YOUR CODE HERE
    raise NotImplementedError()
    """TASK 1.1: END"""

In [ ]:
if not AUTOGRADING:
    model_theta_scripted = torch.jit.script(model_theta)
    model_theta_scripted.save(str(statedicts_dir / "task_1-1_model_theta.pt"))
    
    evaluate_model(
        model_theta,
        test_dataset_theta,
        model="theta",
        file="task_1-1_model_theta_prediction_error.pdf",
    )

### 1.1 - Analyse Model performance (4p)
Analyze the prediction accuracy of the model on the test data from heat map of the loss outputted from the `evaluate_model` function. Where does this model have the lowest accuracy? What could be an explanation for the loss of accuracy in those regions? **2p** (Answer in the cell below)

In general, a separate test dataset is used to evaluate a trained model. Why? **(2p)** (Answer in the cell below)

## Task 1.2: Indirectly predict the angle
We are going to improve the accuracy by pre-processing the target data. Specifically, we will create a model $M_{trig}$ that learns to predict $\sin(\theta)$ and $\cos(\theta)$ for both links instead of directly predicting $\theta$. Then, we can use the trigonometric relation to retrieve an estimate of $\theta$ for both links.

Note: In practice you would use the `atan2` implementation, as the regular arctangent only covers $[-\frac{1}{2}\pi, \frac{1}{2}\pi]$} $\theta=\arctan(\frac{\sin(\theta)}{\cos(\theta)})$ 

### Prepare the data (0p)
Run the functions below to load the data generated in `task_1a_generate_data.ipynb`, split the training into train/validation and put them in a dataloader. 

In [ ]:
if not AUTOGRADING:
    train_loader_trig, val_loader_trig = load_training_dataset_dataloader(
        datasets_dir / "dataset_double_pendulum_train.npz",
        _rng=init_rng,
        model_type="trig",
    )
    test_loader_trig, test_dataset_trig = load_test_dataset_dataloader(
        datasets_dir / "dataset_double_pendulum_test.npz",
        _rng=init_rng,
        model_type="trig",
    )

### 1.2 - Create the model (1.5p)

Copy the model you created in Task 1.1. Change the number of hidden units of the final layer from 2 to 4. We do so, because we now want to predict two outputs ($\sin(\theta)$, $\cos(\theta)$) for both link angles, instead of only two outputs, $\theta_1$ and $\theta_2$.

In [ ]:
""" TASK 1.2: CREATE MODEL HERE """
# YOUR CODE HERE
raise NotImplementedError()
"""TASK1.2: END"""

model_trig = NeuralNetworkTrig()
total_params_trig = sum(p.numel() for p in model_trig.parameters())
print("Total number of neural network parameters: ", total_params_trig)

### 1.2 - Train the model (2p)

Train the model with the same training parameters defined below as in task 1.1 but using the training data `train_loader_trig`. Do this 10 times and record the prediction error of the final model of each of the 10 runs on the test dataset `test_loader_theta` using the function `evaluate_model_test_data_trig`. Don't forget to reinitialize the parameters on each run! **hint:** reduce the number of epochs and lower the number of runs to `1` while getting your model working. 

In [ ]:
# Training parameters
optimizer = torch.optim.SGD(model_trig.parameters(), lr=1e-3)
loss_fn = torch.nn.MSELoss()
error_fn = torch.nn.L1Loss()
num_epochs = 100


def evaluate_model_test_data_trig(_model, _test_loader):
    running_loss_test = 0.0
    running_error_test = 0.0
    count = 0
    with torch.no_grad():
        for i, data_test in enumerate(_test_loader):
            count += 1
            inputs_test, labels_test = data_test
            batch = labels_test.shape[0]
            est = _model(inputs_test)
            est_th = trig_to_theta(est)
            label_th = trig_to_theta(labels_test)
            loss_test = loss_fn(est_th, label_th)
            error_test = error_fn(est_th, label_th)
            running_loss_test += loss_test.item()
            running_error_test += error_test.item()
    print(
        f"Loss on test data: {running_loss_test / count:.3f}, Average prediction error of model on test data: {running_error_test / count:.3f}"
    )

    #     print(f'test loss: {running_loss_train / count_train:.3f}, average test prediction error: {running_error_test / count_train:.3f}')
    return running_loss_test / count

In [ ]:
if not AUTOGRADING:
    """TASK 1.2: TRAIN MODEL HERE"""
    # YOUR CODE HERE
    raise NotImplementedError()
    """TASK 1.2: END"""

In [ ]:
if not AUTOGRADING:
    model_trig_scripted = torch.jit.script(model_trig)
    model_trig_scripted.save(str(statedicts_dir / "task_1-2_model_trig.pt"))
    
    evaluate_model(
        model_trig,
        test_dataset_trig,
        model="trig",
        file="task_1-2_model_trig_prediction_error.pdf",
    )

### 1.2 - Analyse Model performance (4p)
Compare the prediction estimates for $M_{trig}$ with the plot for $M_θ$. Why does indirectly predicting the angle improve the prediction accuracy? **(2p)** (Answer in the cell below)

Why is it not sufficient to predict only sin(θ) and use its inverse θ = arcsin(sin(θ)) to get an estimate of the angle? **(2p)**

## Task 1.3: Indirectly predict the angles with a Convolutional Neural Network (10)
Instead of using a vanilla fully-connected neural network, we will build a prediction model $M_{cnn}$ that uses a convolutional neural network (CNN). 

### 1.3 - Prepare the data
Run the functions below to load the data generated in `task_1a_generate_data.ipynb`, split the training into train/validation and put them in a dataloader. 


In [ ]:
if not AUTOGRADING:
    train_loader_cnn, val_loader_cnn = load_training_dataset_dataloader(
        datasets_dir / "dataset_double_pendulum_train.npz",
        _rng=init_rng,
        model_type="cnn",
    )
    test_loader_cnn, test_dataset_cnn = load_test_dataset_dataloader(
        datasets_dir / "dataset_double_pendulum_test.npz",
        _rng=init_rng,
        model_type="cnn",
    )

### 1.3 - Create the model (2p)
Again, create a sequential PyTorch model with `torch.nn.Sequential([...])` and define the following architecture with Convolutional and pooling layers. 
- Start with a CNN layer `torch.nn.Conv2d(...)` with 32 filters, kernel size of `3x3` and a ReLU activation function.
- Add another convolutional layer with 10 filters
- Then add a max pooling layer `torch.nn.MaxPool2d(...)` with a pool size of `2x2`.
- Flatten the output of the pooling layer
- Add a fully connected layer of 30 with a relu activation function.
- Finally, add a fully connected layer without activation. Remember, the number of ouput units must match the dimension of the target data, which is 4 as we will predict the trigonometric function again. 

In [ ]:
"""TASK 1.3: CREATE MODEL HERE"""
# YOUR CODE HERE
raise NotImplementedError()
"""TASK 1.3: END HERE"""

model_cnn = NeuralNetworkCNN()
total_params_cnn = sum(p.numel() for p in model_cnn.parameters())
print("total number of model parameters: ", total_params_cnn)

### 1.3 - train the model (2p)

Train the model using the same training parameters used in tasks 1.1 and 1.2. 

Train the model with the same training parameters defined below as in task 1.1 and 1.2 but using the training data `train_loader_cnn`. Do this 10 times and record the prediction error of the final model of each of the 10 runs on the test dataset `test_loader_theta` using the function `evaluate_model_test_data_cnn`. Don't forget to reinitialize the parameters on each run! **hint:** reduce the number of epochs and lower the number of runs to `1` while getting your model working. 

In [ ]:
# Training parameters
optimizer = torch.optim.SGD(model_cnn.parameters(), lr=1e-3)
loss_fn = torch.nn.MSELoss()
error_fn = torch.nn.L1Loss()
num_epochs = 100


def evaluate_model_test_data_cnn(_model, _test_loader):
    """
    evaluates the model_cnn on the
    """
    running_loss_test = 0.0
    running_error_test = 0.0
    count = 0
    with torch.no_grad():
        for i, data_test in enumerate(_test_loader):
            count += 1
            inputs_test, labels_test = data_test
            #             inputs_test = torch.permute(inputs_test, (0, 3, 1, 2))
            batch = labels_test.shape[0]
            est = _model(inputs_test)
            est_th = trig_to_theta(est)
            label_th = trig_to_theta(labels_test)
            loss_test = loss_fn(est_th, label_th)
            error_test = error_fn(est_th, label_th)
            running_loss_test += loss_test.item()
            running_error_test += error_test.item()

    print(
        f"Loss on test data: {running_loss_test / count:.3f}, Average prediction error of model on test data: {running_error_test / count:.3f}"
    )
    return running_error_test / count

In [ ]:
if not AUTOGRADING:
    """TASK 1.3: TRAIN MODEL HERE"""
    # YOUR CODE HERE
    raise NotImplementedError()
    """TASK 1.3: END"""

In [ ]:
if not AUTOGRADING:
    model_cnn_scripted = torch.jit.script(model_cnn)
    model_cnn_scripted.save(str(statedicts_dir / "task_1-3_model_cnn.pt"))
    
    evaluate_model(
        model_cnn,
        test_dataset_cnn,
        model="cnn",
        file="task_1-3_model_cnn_prediction_error.pdf",
    )

### 1.3 - Analyse and Compare Model performance 
Make a comparison of the different models (i.e. M θ, M trig, M cnn) based on the average prediction accuracy
on the test dataset and the number of trainable parameters. Which model would you prefer and why? (answer in the cell below)

Describe the task here!

If you change the activation of the last fully connected layer to ReLU, the prediction accuracy completely deteriorates. Why?

Describe the task here!

## Task 1.4: Variance across runs (2.5p) 
In the previous tasks, the prediction accuracy varied across different runs even though the underlying code and dataset remained unchanged.

Use the code cell below to set the seed for PyTorch. Train the model for a seed of 0 and then for seed of 1, recording the average prediction accuracy and standard deviation for both. 

In [ ]:
torch.manual_seed(0)  # set seed to either 0 or 1

Copy the architecture of $M_{cnn}$ to create a new model below. Don't forget to comment on the code to set the seed again when working on the other tasks.

In [ ]:
""" TASK 1.4: CREATE MODEL HERE """
# YOUR CODE HERE
raise NotImplementedError()
"""TASK1.4: END"""
model_theta_seed = NeuralNetworkThetaSeed()

Set up a training loop as done for the task 1.3 and run this 10 times while having the seed value as 0 and then run it again for a seed of 1. Use the same datasets as in task 1.3. **(0.5p)**. 

In [ ]:
# Training parameters
optimizer = torch.optim.SGD(model_theta_seed.parameters(), lr=1e-3)
# We choose the mean square loss as it is a regression problem
loss_fn = torch.nn.MSELoss()
num_epochs = 30

In [ ]:
if not AUTOGRADING:
    losses_seed0 = []
    losses_seed1 = []
    """ TASK 1.4: TRAIN MODEL HERE """
    # YOUR CODE HERE
    raise NotImplementedError()
    """TASK 1.4: END"""

In [ ]:
if not AUTOGRADING:
    model_cnn_scripted = torch.jit.script(model_cnn)
    model_cnn_scripted.save(str(statedicts_dir / "task_1-4_model_cnn.pt"))
    
    evaluate_model(
        model_cnn,
        test_dataset_cnn,
        model="cnn",
        file="task_1-4_model_cnn_prediction_error.pdf",
    )

- What do you observe? (1p)

- What is the benefit of seeding the pseudo-random generator in practice? **(1p)**

Write your answers in the cell below

### One last thing!
A neural network that can predict angles is required for task 2b. Therefore, with the model seeded, you are free to change the training parameters such as learning rate, number of epochs etc. to see how far you can increase the prediction accuracy of link angles. Alternatively, you can just use the standard model trained in 1.4 but be warned the accuracy of your model will affect the performance of you controller in task 2b.

## Task 1.5: Spiking neural networks (15p)

If our goal is to control the double pendulum, the angular velocity $\dot{\theta}$ is generally also required. However, temporal information cannot be extracted from individual images. We are going to utilize the temporal advantage of the spiking neural networks (SNN) to predict the angular velocity of each link. Instead of static individual images, we adopt event-based data as the input.

### 1.5 - Warm Up
Use the Jupyter notebook `task_1-5_SNN_warmup.ipynb` to understand the neuron structure of SNNs, construct a LIF neuron model and a simple fully connected SNN.

### 1.5 - Prepare the dataset

In [ ]:
# This part has to be same as task_1a_generate_data.ipynb
train_th1_range = jnp.arange(-jnp.pi / 6.0, jnp.pi / 6.0, jnp.pi / 30.0)
train_th2_range = jnp.arange(-jnp.pi, jnp.pi, jnp.pi / 6.0)
test_th1_range = jnp.arange(-jnp.pi / 6.0, jnp.pi / 6.0, jnp.pi / 30.0)
test_th2_range = jnp.arange(-jnp.pi, jnp.pi, jnp.pi / 6.0)

TRAIN_NUM_DATA = len(train_th1_range) * len(train_th2_range)
TEST_NUM_DATA = len(test_th1_range) * len(test_th2_range)
NUM_SNN_DATA = 5

In [ ]:
if not AUTOGRADING:
    train_set_snn = SNNDataset(
        datasets_dir / "event_based_data" / "train",
        TRAIN_NUM_DATA,
        NUM_SNN_DATA,
    )
    train_loader_snn = DataLoader(train_set_snn, batch_size=16, shuffle=True, drop_last=True)

    test_set_snn = SNNDataset(
        datasets_dir / "event_based_data" / "test",
        TEST_NUM_DATA,
        NUM_SNN_DATA,
    )
    test_loader_snn = DataLoader(test_set_snn, batch_size=16, shuffle=True, drop_last=True)

### 1.5 - Create the model (4p)

We create a class called `snnModel` extending `nn.Module`. Please fill this class to implement the model of SNNs. We adopt the LIF model for neurons. \\

* Start with a convolutional layer with 16 as the number of output channels, kernel size of `5x5`.
* Then, add a max pooling layer with a pooling size of `4x4`.
* Add the second convolutional layer with 16 as output channel size, kernel size of `3x3`.
* Add the third convolutional layer with 16 as output channel size, kernel size of `3x3`.
* All neurons use LIF model, and obtain the membrane potential in a recurrent way.
* Flatten the membrane potential in the last layer and feed it into a non-linear layer with traditional neurons with ReLU as the activation function. The number must match the dimension of our target data which is 2.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
"""TASK1.5: END"""

model_snn = snnModel()

### 1.5 - Train the model (3p)

Train the SNN model in the same way with other models.

In [ ]:
optimizer = torch.optim.Adam(model_snn.parameters(), lr=1e-3)
loss_fn = torch.nn.MSELoss()
num_epochs = 200

In [ ]:
onp.random.seed(42)
torch.manual_seed(42)
if not AUTOGRADING:
"""TASK 1.5: TRAIN MODEL HERE"""
# YOUR CODE HERE
raise NotImplementedError()
"""TASK 1.5: END"""

In [ ]:
# Evaluate the SNN model with test data
if not AUTOGRADING:
    with torch.no_grad():
        model_snn = model_snn.eval()
        print_loss = 0.0
        count = 0
        for i, (events, targets) in enumerate(test_loader_snn):
            output = model_snn(events)
            loss = loss_fn(output, targets)
            print_loss += loss.item()
            count = count + 1

        print_loss = print_loss / count
        print(f"The MSE loss of the model on the test dataset is: {print_loss}")

### 1.5 - Analysis of results (8p)

Does your loss decrease step by step? During the training process, is the training speed faster or slower comparing to CNN and why? Please analyse the advantages and disadvantages of SNN. **(8p)**